In [49]:
import numpy as np
import pandas as pd

In [48]:
data= pd.read_csv("/content/drive/MyDrive/Science Journal/data.csv")
print(data)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [57]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

le = LabelEncoder()
y = le.fit_transform(data['Category'])

X_train, X_test, y_train, y_test = train_test_split(data['Message'], data['Category'], test_size=0.2, random_state=42)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_sequence_length = 100
X_train_seq = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_seq = pad_sequences(X_test_seq, maxlen=max_sequence_length)

In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


embedding_size = 100  
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, embedding_size, input_length=max_sequence_length))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(patience=3, monitor='val_loss', mode='min', restore_best_weights=True)

batch_size=32
epochs=10
# model.fit(X_train_seq, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_seq, y_test))
history = model.fit(X_train_seq, y_train, batch_size=batch_size, epochs=epochs,
                    validation_data=(X_test_seq, y_test), callbacks=[early_stopping])

loss, accuracy = model.evaluate(X_test_seq, y_test, batch_size=batch_size)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/10
140/140 [==============================] - 43s 277ms/step - loss: 0.1361 - accuracy: 0.9560 - val_loss: 0.0537 - val_accuracy: 0.9874
Epoch 2/10
140/140 [==============================] - 33s 237ms/step - loss: 0.0238 - accuracy: 0.9935 - val_loss: 0.0446 - val_accuracy: 0.9892
Epoch 3/10
140/140 [==============================] - 38s 269ms/step - loss: 0.0065 - accuracy: 0.9989 - val_loss: 0.0503 - val_accuracy: 0.9901
Epoch 4/10
140/140 [==============================] - 37s 264ms/step - loss: 0.0029 - accuracy: 0.9989 - val_loss: 0.0462 - val_accuracy: 0.9901
Epoch 5/10
35/35 [==============================] - 2s 53ms/step - loss: 0.0446 - accuracy: 0.9892
Test Loss: 0.044642310589551926
Test Accuracy: 0.9892376661300659


In [54]:
inputText = input("Enter the text: ")

sequence = tokenizer.texts_to_sequences([inputText])
sequence = pad_sequences(sequence, maxlen=max_sequence_length)

res = model.predict(sequence)[0][0]

if res >= 0.5:
    print("SPAM")
else:
    print("NOT SPAM")

Enter the text: asdfasdf
1/1 [==============================] - 0s 33ms/step
NOT SPAM
